<a href="https://colab.research.google.com/github/RaulOAlves/NDWI_Codajas/blob/main/NDWI_AMBIGEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap earthengine-api -q

In [2]:
#Importando as bibliotecas
import geemap
import ee

In [ ]:
#Autenticação e inicialização
ee.Authenticate()
ee.Initialize()

In [4]:
#Definindo a região de interesse
roi = ee.Geometry.Point([-62.25579998379851, -3.7898888317585677]);

In [10]:
#Definir coleção de imagens
collection_l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
                              .filterBounds(roi)\
                              .filterDate("1990-01-01","2023-01-01")\
                              .filter(ee.Filter.lt('CLOUD_COVER',1))
print('Landsat 8 - Qtds', collection_l8.size().getInfo())

Landsat 8 - Qtds 10


In [16]:
#Criar uma função para calcular o NDWI

def ndwi(img):
  #Fator de escala bandas opticas
  opticalBands = img.select('SR_B.').multiply(0.0000275).add(-0.2)
  #Fator de escala bandas termais
  thermalBands = img.select('ST_B.*').multiply(0.00341802).add(149.0)
  #NDWI
  Ndwi_image = img.normalizedDifference(['SR_B3','SR_B5']).rename('NDWI')
  #Aplicando e adicionando as bandas com fator de escala
  return img.addBands(opticalBands, None, True).addBands(thermalBands, None, True)\
         .addBands(Ndwi_image).copyProperties(img,['system:time_start'])\
         .set('date', img.date().format('YYYY-MM-dd'))

In [19]:
#Aplicando o fator de escala e NDWI em todas as imagens
collection_l8_ndwi = collection_l8.map(ndwi)

In [31]:
#Criando as mascaras para identificar valor máximo e médio
mask_max = collection_l8_ndwi.select('NDWI').max().gt(0).selfMask()
mask_mean = collection_l8_ndwi.select('NDWI').mean().gt(0).selfMask()
mask_min = collection_l8_ndwi.select('NDWI').min().gt(0).selfMask()

In [32]:
#Definindo um método de exibição dos blocos de iamgens do gee
Map = geemap.Map()
#Centralizer a imagem
Map = geemap.Map(location=[-62.25579998379851, 3.7898888317585677], zoom_start = 10)
#Para visualizar a imagem de satélite do google como mapa base
#Map.add_basemap('HYBRID')
Map.add_ee_layer(collection_l8_ndwi.median(), {'bands':['SR_B4','SR_B3','SR_B2'], 'min': 0.01, 'max': 0.1}, 'RGB')
#Adicionando as mascaras
Map.add_ee_layer(mask_max,{'palette':['blue'], 'min':0,'max':1}, 'NDWI max')
Map.add_ee_layer(mask_mean,{'palette':['cyan'], 'min':0,'max':1}, 'NDWI mean')
Map.add_ee_layer(mask_min,{'palette':['red'], 'min':0,'max':1}, 'NDWI min')
display(Map)

Map(center=[-62.25579998379851, 3.7898888317585677], controls=(WidgetControl(options=['position', 'transparent…